In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
dataset = pd.read_csv('final_dataset_3.csv')

dataset = dataset.set_index('Unnamed: 0')

print(dataset.columns)
drop_columns = ['eda_time','form_time','form_time.1', 'gender']
dataset.drop(drop_columns, axis=1, inplace=True)
dataset = dataset.rename(columns={"perc_change": "eda_perc_change", "abs_change": "eda_abs_change"})
dataset.sort_values(by='participantID', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)




Index(['form_time', 'eda_time', 'eda_perc_change', 'perc_bpm', 'abs_bpm',
       'perc_ibi', 'abs_ibi', 'perc_sdnn', 'abs_sdnn', 'perc_sdsd', 'abs_sdsd',
       'perc_rmssd', 'abs_rmssd', 'perc_pnn20', 'abs_pnn20', 'perc_pnn50',
       'abs_pnn50', 'perc_hr_mad', 'abs_hr_mad', 'perc_sd1', 'abs_sd1',
       'perc_sd2', 'abs_sd2', 'perc_s', 'abs_s', 'perc_sd1/sd2', 'abs_sd1/sd2',
       'perc_breathingrate', 'abs_breathingrate', 'form_time.1',
       'participantID', 'gender', 'change_in_stressed', 'change_in_amused',
       'change_in_nervous', 'change_in_engaged', 'change_in_lost',
       'change_in_disappointed', 'change_in_excited', 'change_in_frustrated',
       'change_in_contempt'],
      dtype='object')


,eda_perc_change,perc_bpm,abs_bpm,perc_ibi,abs_ibi,perc_sdnn,abs_sdnn,perc_sdsd,abs_sdsd,perc_rmssd,...,participantID,change_in_stressed,change_in_amused,change_in_nervous,change_in_engaged,change_in_lost,change_in_disappointed,change_in_excited,change_in_frustrated,change_in_contempt
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2,-18.175823,1.233378,17.911896,0.810781,-147.922511,1.243654,29.718330,1.474264,35.313478,1.457498,...,0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
4,12.059950,0.946140,-4.789594,1.056926,38.409091,1.415998,8.794376,1.209645,3.075595,1.223343,...,1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
13,5.554695,1.026443,2.120372,0.974238,-19.276820,0.613986,-56.181970,0.427218,-78.144997,0.426255,...,1,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,-11.869750,1.259298,13.986117,0.794093,-229.046474,2.980787,101.935473,2.251378,45.240920,2.582955,...,2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10,10.470302,1.054041,3.543500,0.948730,-46.914062,0.875804,-9.381374,0.209045,-64.717699,0.300858,...,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
12,-16.508557,0.856132,-14.880884,1.168044,97.478693,1.916190,28.123171,2.016577,32.726063,1.764414,...,3,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
15,33.068751,1.500757,32.379201,0.666330,-309.619816,1.380636,38.622936,2.451917,74.019488,2.595382,...,3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,57.804164,1.233832,14.670179,0.810483,-181.246151,1.226543,16.921944,0.588983,-34.886387,0.838645,...,4,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
14,9.757678,1.011955,1.011207,0.988186,-8.380682,1.551937,43.668503,2.664646,50.969177,1.590097,...,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
perc_columns = [col for col in dataset.columns if 'perc' in col]
dataset_perc = df = dataset[dataset.columns.drop(list(dataset.filter(regex='abs')))]

In [7]:
columns = ['eda_perc_change','perc_bpm','perc_rmssd','perc_sdsd','participantID', 'change_in_stressed', 'change_in_amused','change_in_engaged','change_in_frustrated']
final_dataset = dataset_perc#[columns]
final_dataset[final_dataset == -1] = 0
final_dataset['eda_perc_change'] = final_dataset['eda_perc_change']/100 +1

/Users/sachin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sachin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3718: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
/Users/sachin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [8]:
final_dataset

,eda_perc_change,perc_bpm,perc_ibi,perc_sdnn,perc_sdsd,perc_rmssd,perc_pnn20,perc_pnn50,perc_hr_mad,perc_sd1,...,participantID,change_in_stressed,change_in_amused,change_in_nervous,change_in_engaged,change_in_lost,change_in_disappointed,change_in_excited,change_in_frustrated,change_in_contempt
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,1.191747,1.105372,0.904673,0.758386,0.639516,0.528432,0.652632,0.388471,0.400000,0.524149,...,8,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.634600,0.905580,1.104264,1.913173,1.494328,1.810519,1.428571,2.666667,2.000000,1.836590,...,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.818242,1.233378,0.810781,1.243654,1.474264,1.457498,0.921053,1.246964,1.750000,1.466168,...,0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
3,0.881303,1.259298,0.794093,2.980787,2.251378,2.582955,1.000000,1.470588,1.714286,2.584538,...,2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.120599,0.946140,1.056926,1.415998,1.209645,1.223343,1.240385,1.791667,1.000000,1.214502,...,1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,1.731810,1.276181,0.783588,1.665727,2.400520,3.334103,1.944444,8.750000,2.666667,3.333646,...,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.669397,1.036094,0.965163,3.190159,2.624728,3.570906,1.191067,2.115385,5.500000,3.566602,...,9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.870468,0.909589,1.099398,0.769442,1.553925,1.968773,1.500000,1.250000,0.642857,3.278417,...,7,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
8,1.578042,1.233832,0.810483,1.226543,0.588983,0.838645,1.757396,1.780220,1.333333,0.842642,...,4,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [9]:
def train_loop(model,x_feature):
    
    if model == "svm":
        clf = SVC(kernel='linear')
    elif model == "dt":
        clf = DecisionTreeClassifier(random_state=0)
    elif model == "rf":
        clf = RandomForestClassifier(n_estimators = 10,max_depth=2, random_state=0,min_samples_leaf=1)
    elif model == 'knn':
        clf = KNeighborsClassifier(n_neighbors = 3)
        
        
    y_labels = ['change_in_stressed', 'change_in_amused','change_in_engaged','change_in_frustrated']
    for j in range(len(y_labels)):
        print(y_labels[j])


        participantIDs = np.arange(11)
        total_correct = 0
        train_total_points = 0
        train_total_correct = 0
        total_points = 0
        total_correct_pred_0 = 0
        total_correct_pred_1 = 0
        for i, a in enumerate(participantIDs):
            #print("participantID",i)
            train_ids = participantIDs[participantIDs!=i]
            test_ids = participantIDs[participantIDs==i]
            total_train_data = final_dataset.loc[final_dataset['participantID'].isin(train_ids)]
            test_data = final_dataset.loc[final_dataset['participantID'].isin(test_ids)]

 

            x_train = total_train_data[x_feature]

            x_test = test_data[x_feature] 


            y_train = total_train_data[y_labels[j]]
            y_test = test_data[y_labels[j]]


            clf.fit(x_train, y_train)
            predict = clf.predict(x_test)
            accuracy = predict == y_test
            total_correct += sum(accuracy)
            #print("total_correct", total_correct)
            total_points += len(y_test)
            train_total_points += len(y_train)
            print("prediction:",predict)
            print("true:", y_test.values )


            train_accuracy = clf.predict(x_train) == y_train

            train_total_correct += sum(train_accuracy)
            #print("total_correct", total_correct)


            total_correct_pred_0 += sum(y_test==0)
            total_correct_pred_1 += sum(y_test==1)
            pred_0 = 0
            #print("total_points", total_points)
            #print(sum(accuracy))
            #print("The train accuracy:", clf.score(x_train, y_train))
            #print("The test accuracy:", clf.score(x_test, y_test))

            #print('-'*15)

        print("train_accuracy for", y_labels[j], ": ", train_total_correct/train_total_points)
        print("test_accuracy for", y_labels[j], ": ", total_correct/total_points)
        print("accuracy for", y_labels[j], " predict 0 : ", total_correct_pred_0/total_points)
        print("accuracy for", y_labels[j], " predict 1 : ", total_correct_pred_1/total_points)
        print()
        print()
        print('-'*50)


        
    
        
    
    

In [12]:
train_loop("dt",['eda_perc_change','perc_sdsd','perc_bpm'])

change_in_stressed
prediction: [1.]
true: [1.]
prediction: [1. 0.]
true: [1. 1.]
prediction: [1.]
true: [1.]
prediction: [1. 1. 0.]
true: [0. 1. 1.]
prediction: [0. 1.]
true: [1. 0.]
prediction: [1.]
true: [0.]
prediction: [1.]
true: [0.]
prediction: [1. 1.]
true: [1. 0.]
prediction: [1.]
true: [1.]
prediction: [0.]
true: [1.]
prediction: [1.]
true: [1.]
train_accuracy for change_in_stressed :  1.0
test_accuracy for change_in_stressed :  0.4375
accuracy for change_in_stressed  predict 0 :  0.3125
accuracy for change_in_stressed  predict 1 :  0.6875


--------------------------------------------------
change_in_amused
prediction: [1.]
true: [1.]
prediction: [0. 1.]
true: [1. 1.]
prediction: [1.]
true: [1.]
prediction: [1. 0. 0.]
true: [0. 1. 0.]
prediction: [0. 1.]
true: [1. 1.]
prediction: [1.]
true: [0.]
prediction: [1.]
true: [0.]
prediction: [1. 1.]
true: [1. 1.]
prediction: [1.]
true: [0.]
prediction: [0.]
true: [1.]
prediction: [1.]
true: [0.]
train_accuracy for change_in_amused :